In [1]:
import tensorflow as tf
import cPickle

In [2]:
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

Hello, TensorFlow!


In [3]:

# Import Numpy, TensorFlow, TFLearn, and MNIST data
import numpy as np
import tensorflow as tf
import tflearn
import tflearn.datasets.mnist as mnist

In [4]:
from sklearn.utils.testing import assert_array_equal

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
import numpy as np
import pandas as pd
# from rpropClassfier import RPClassifier

In [28]:
from __future__ import division
import numpy as np

from sklearn.base import BaseEstimator, ClassifierMixin

from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import RPropMinusTrainer
from pybrain.datasets.supervised import SupervisedDataSet

from math import sqrt

class RPClassifier(BaseEstimator, ClassifierMixin):
    """A network classifier with RPropMinusTrainer in PYbrain
    prarameters
    -----------
    h_size : interger, default = 2.
        Number of neurons in hidden layer.
    epo :  interger, default = 2.
       Train on the current dataset for the given number of epochs.
    verbose: interger, default = 2
            Controls the verbosity of the network building process.
    Attributes
    ----------
    in_size : interger.
        The number of input array columns. This will be number of
        neurons in input layer.
    out_size : interger
        The number of label arrya columns. this will be the number of
        neurons in output layer.
    net : the network.
    references
    ----------
    PyBrain document
    http://pybrain.org/docs/index.html
    """

    def __init__(self, h_size=2, epo=2, verbose=0):
        self.h_size = h_size
        self.epo = epo
        self.verbose = verbose
        pass

    def fit(self, X, y):
        """ build a network from training set (X, y).
        parameters
        ----------
        X : array-like or sparse matrix of shape = [n_samples, n_features]
        y : array-like, shape = [n_samples] or [n_samples, n_output]
        return
        ------
        self : object
        """

        y_train = np.array( y)
        _, self.in_size = X.shape
        _, self.out_size = y_train.shape

        ds = SupervisedDataSet(self.in_size, self.out_size)

        ds.setField('input', X)
        ds.setField('target', y_train)

        self.net = buildNetwork(self.in_size,
                                self.h_size, self.out_size, bias=True)
        trainer = RPropMinusTrainer(self.net, dataset=ds)

        if self.verbose > 0:
            print ("start training ...")

        for n in xrange(self.epo):
            mse = trainer.train()
            rmse = sqrt(mse)
            if self.verbose > 0:
                print ("RMSE = %8.3f epoch = %d" % (rmse, n))
        return self

    def predict(self, X):
        """Predict class for X.
        Parameters
        ----------
        X : array-like or sparse matrix of shape = [n_samples, n_features]
        Returns
        -------
        y : array-like, shape = [n_samples] or [n_samples, n_output]
        """
        # TODO for multi output classes
        p = self.predict_proba(X)
        p_class = np.array([1 if pn > 0.5 else 0 for pn in p])
        return p_class

    def predict_proba(self, X):
        """Predict class probabilities for X.
        Parameters
        ----------
        X : array-like or sparse matrix of shape = [n_samples, n_features]
        Returns
        -------
        p : array-like, shape = [n_samples] or [n_samples, n_output]
        """

        row_size, in_size = X.shape

        y_test_dumy = np.zeros([row_size, self.out_size])

        # check size
        assert(self.net.indim == in_size)

        ds = SupervisedDataSet(in_size, self.out_size)

        ds.setField('input', X)
        ds.setField('target', y_test_dumy)

        p = self.net.activateOnDataset(ds)
        return np.array(p)

In [29]:
fileName = "ttt_data_for_tf.pkl"

In [30]:
with open(fileName, "rb") as input_file:
    data = cPickle.load(input_file)

In [31]:

# Retrieve the training and test data
trainX0, trainY0= data

In [32]:
trainX = np.asarray([list(x) for x in trainX0])
trainX[:4]

array([[ 0, -1,  1,  1,  0, -1, -1,  0,  1],
       [ 1,  0,  1, -1,  1,  0, -1, -1,  0],
       [-1,  1, -1,  0, -1,  1, -1,  0, -1],
       [-1,  1, -1, -1,  1,  0,  0, -1, -1]])

In [33]:
trainX =  np.apply_along_axis(lambda x: (x+1.0)/2.0, -1,trainX)

In [34]:
trainY = np.asarray(trainY0)

In [35]:
trainX.shape[1]

9

In [36]:
trainX

array([[ 0.5,  0. ,  1. , ...,  0. ,  0.5,  1. ],
       [ 1. ,  0.5,  1. , ...,  0. ,  0. ,  0.5],
       [ 0. ,  1. ,  0. , ...,  0. ,  0.5,  0. ],
       ..., 
       [ 0.5,  0.5,  0. , ...,  0. ,  1. ,  1. ],
       [ 0.5,  0. ,  1. , ...,  0. ,  0.5,  0. ],
       [ 1. ,  1. ,  0. , ...,  0.5,  0.5,  0. ]])

In [37]:
trainY

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [60]:
bpc = RPClassifier(h_size=10, epo=100,verbose=1)
bpc.fit(trainX, trainY)


start training ...
RMSE =    1.159 epoch = 0
RMSE =    0.801 epoch = 1
RMSE =    0.622 epoch = 2
RMSE =    0.516 epoch = 3
RMSE =    0.398 epoch = 4
RMSE =    0.394 epoch = 5
RMSE =    0.348 epoch = 6
RMSE =    0.352 epoch = 7
RMSE =    0.309 epoch = 8
RMSE =    0.289 epoch = 9
RMSE =    0.282 epoch = 10
RMSE =    0.269 epoch = 11
RMSE =    0.261 epoch = 12
RMSE =    0.257 epoch = 13
RMSE =    0.246 epoch = 14
RMSE =    0.245 epoch = 15
RMSE =    0.242 epoch = 16
RMSE =    0.237 epoch = 17
RMSE =    0.233 epoch = 18
RMSE =    0.229 epoch = 19
RMSE =    0.228 epoch = 20
RMSE =    0.227 epoch = 21
RMSE =    0.224 epoch = 22
RMSE =    0.223 epoch = 23
RMSE =    0.221 epoch = 24
RMSE =    0.219 epoch = 25
RMSE =    0.217 epoch = 26
RMSE =    0.216 epoch = 27
RMSE =    0.216 epoch = 28
RMSE =    0.215 epoch = 29
RMSE =    0.214 epoch = 30
RMSE =    0.213 epoch = 31
RMSE =    0.213 epoch = 32
RMSE =    0.212 epoch = 33
RMSE =    0.212 epoch = 34
RMSE =    0.212 epoch = 35
RMSE =    0.211 epo

RPClassifier(epo=100, h_size=10, verbose=1)

In [61]:
trainX[0]

array([ 0.5,  0. ,  1. ,  1. ,  0.5,  0. ,  0. ,  0.5,  1. ])

In [62]:
p = bpc.predict_proba(trainX)

In [63]:
actual = trainY.argmax(axis=1)
predictions = p.argmax(axis=1)

In [64]:
actual

array([1, 6, 6, ..., 3, 3, 2])

In [65]:
predictions

array([6, 6, 0, ..., 2, 6, 4])

In [66]:
test_accuracy = np.mean(predictions == actual, axis=0)

# Print out the result
print("Test accuracy: ", test_accuracy)

('Test accuracy: ', 0.42816028337392076)


In [ ]:
# Define the neural network
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    # Inputs
    net = tflearn.input_data([None, trainX.shape[1]],dtype=tf.float32)

    # Hidden layer(s)
    net = tflearn.fully_connected(net, 10, activation='ReLU')

    # Output layer and training model
    net = tflearn.fully_connected(net, trainY.shape[1], activation='softmax')
    net = tflearn.regression(net, optimizer='adam', learning_rate=0.01, loss='categorical_crossentropy')
    # softmax
    # ReLU
    # categorical_crossentropy
    #sgd adam
    model = tflearn.DNN(net)
    return model

In [ ]:

# Build the model
model = build_model()

In [ ]:
# Training
model.fit(trainX, trainY, validation_set=0.01, show_metric=True, batch_size=5, n_epoch=5)

In [ ]:
# Compare the labels that our model predicts with the actual labels

# Find the indices of the most confident prediction for each item. That tells us the predicted digit for that sample.
predictions = np.array(model.predict(trainX)).argmax(axis=1)

# Calculate the accuracy, which is the percentage of times the predicated labels matched the actual labels
actual = trainY.argmax(axis=1)
test_accuracy = np.mean(predictions == actual, axis=0)

# Print out the result
print("Test accuracy: ", test_accuracy)

In [ ]:
n=200
print trainX[n]
print trainY[n]
model.predict([trainX[n]])

In [ ]:
model.predict([trainX[1000]])

In [ ]:
predictions